## Monitoramento de Ações com Bot do Telegram

#### Install:

In [0]:
!pip install requests

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0e980f5e-2f02-4dd3-b5db-8be92bf65c8f/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install yfinance

     |████████████████████████████████| 94 kB 2.2 MB/s 
     |██████████████

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 2.9 MB 39.1 MB/s 
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - error
  ERROR: Command errored out with exit status 1:
   command: /local_disk0/.ephemeral_nfs/envs/pythonEnv-0e980f5e-2f02-4dd3-b5db-8be92bf65c8f/bin/python /local_disk0/.ephemeral_nfs/envs/pythonEnv-0e980f5e-2f02-4dd3-b5db-8be92bf65c8f/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py get_requires_for_build_wheel /tmp/tmpkyr5zsdk
       cwd: /tmp/pip-install-6_vrpp4k/peewee_57c33a4905564922b9f571cbc02202d8
  Complete output (40 lines):
  
  Error compiling Cython file:
  ------------------------------------------------------------
  ...
      def autocommit(self):
          if self.conn.initialized and self.conn.db:
              return sqlite3_get_autocommit(self.co

#### Imports:

In [0]:
import pandas as pd
import requests
import json
import yfinance as yf

#### Config:

In [0]:
url = "https://api.telegram.org/bot8014629563:AAGNDLxR8HPnNsXLjKHuqFBzdY4t4x4fLp4/sendMessage"


config = [
    {
        "acao": "ITUB4.SA",
        "percent_caiu": 1,
        "percent_subiu": 0.15,
        "vl_caiu": 1,
        "vl_subiu": 0.20,
        "ids_chat": [1103869584],
    },
    {
        "acao": "BBDC4.SA",
        "percent_caiu": 0.5,
        "percent_subiu": 0.5,
        "vl_caiu": 1.5,
        "vl_subiu": 0.15,
        "ids_chat": [1103869584],
    },
    {
        "acao": "SANB11.SA",
        "percent_caiu": 1.0,
        "percent_subiu": 0.02,
        "vl_caiu": 2,
        "vl_subiu": 0.05,
        "ids_chat": [1103869584],
    }
]

#### Get ticket:

In [0]:
def get_ticket(p_acao):
    '''
    função que retorna os valores de uma determinada ação na ultima cotação disponível 
    Args: 
        p_acao::string
    Returns:        
        vl_abertura:: float
        vl_fechamento:: float
        vl_variacao::float
        percent_variacao::float
    '''
    
    df = yf.download(p_acao,period='1d')
    
    vl_open = df.iloc[0]['Open']
    vl_close = df.iloc[0]['Close']
    
    #TRATA ERRO NO YFINANCE
    if(vl_open == 0 or vl_close == 0):
        print('problema no yfinance')
        df = yf.download(p_acao,period='2d')
        vl_open = df.iloc[0]['Open']
        vl_close = df.iloc[0]['Close']    



    vl_variacao = vl_close-vl_open    
    pc_variacao = ((vl_close*100)/vl_open)-100 

    return {"vl_abertura":round(vl_open, 2), "vl_fechamento": round(vl_close, 2),"vl_variacao":round(vl_variacao, 2), "percent_variacao" : round(pc_variacao, 2)}

#### Send message:

In [0]:


def send_message_telegram(p_idChat, p_text):
    '''
    Envia mensagem para o Telegram - via avisa_acoes_bot

    Args:
        p_idChat::list -> Código unico da conversa
        p_text::String -> Corpo da mensagem
    '''

    for idchat in p_idChat:

        payload = json.dumps({
        "chat_id": idchat,
        "text": p_text,
        "parse_mode" :"HTML"
        })
        headers = {
        'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)

    return True

#### Flow:

In [0]:


for trigger_alerta in config:
    print(trigger_alerta)

    # variaveis do gatilho
    acao = trigger_alerta.get("acao","")
    trigger_percent_caiu = trigger_alerta.get("percent_caiu",0)
    trigger_percent_subiu = trigger_alerta.get("percent_subiu",0)
    trigger_vl_caiu = trigger_alerta.get("vl_caiu",0)
    trigger_vl_subiu = trigger_alerta.get("vl_subiu",0)
    trigger_ids_chat = trigger_alerta.get("ids_chat",[])


    # busca cotação e valores
    json_cotacao = get_ticket(acao)

    real_vl_abertura = json_cotacao.get('vl_abertura',0)
    real_vl_fechamento = json_cotacao.get('vl_fechamento',0)
    real_vl_variacao = json_cotacao.get('vl_variacao',0)
    real_percent_variacao = json_cotacao.get('percent_variacao',0)

    
    #inicia validações
    
    print('############################')

    # VALIDA SE A AÇÃO CAIU EM %
    if trigger_percent_caiu != 0:
        print(f'Vai validar se a ação {acao} caiu {trigger_percent_caiu}%')
        print(f'A ação {acao} variou {real_percent_variacao}%')

        print(trigger_percent_caiu*-1)
        print(real_percent_variacao)
        
        if (trigger_percent_caiu*-1 >= real_percent_variacao):
        
            print('--->>> Gatilho ativado')
                    
            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> caiu <b>{real_percent_variacao}%</b>  </u> 
                <pre>Trigger ->{trigger_percent_caiu*-1} % de queda </pre>
            
            '''


            send_message_telegram(trigger_ids_chat,mensagem)
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue


    print('############################')

    # VALIDA SE A AÇÃO SUBIU EM %
    if trigger_percent_subiu != 0:
        print(f'Vai validar se a ação {acao} subiu {trigger_percent_subiu}%')
        print(f'A ação {acao} variou {real_percent_variacao}%')

        print(trigger_percent_subiu)
        print(real_percent_variacao)
        
        if (trigger_percent_subiu <= real_percent_variacao):
        
            print('--->>> Gatilho ativado')
        
            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> subiu <b>{real_percent_variacao}%</b>  </u> 
                <pre>Trigger ->{trigger_percent_subiu} % de subida </pre>
            
            '''

            send_message_telegram(trigger_ids_chat,mensagem)   
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue      

    # VALIDA SE A AÇÃO SUBIU EM VALOR
    if trigger_vl_subiu != 0:
        print(f'Vai validar se a ação {acao} subiu R${trigger_vl_subiu}')
        print(f'A ação {acao} variou {real_vl_variacao}%')

        print(trigger_vl_subiu)
        print(real_vl_variacao)
        
        if (trigger_vl_subiu <= real_vl_variacao):
        
            print('--->>> Gatilho ativado')
        
            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> subiu <b> R$ {real_vl_variacao}</b>  </u> 
                <pre>Trigger ->R$ {trigger_vl_subiu} de subida </pre>
            
            '''

            send_message_telegram(trigger_ids_chat,mensagem)   
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue                      

    # VALIDA SE A AÇÃO CAIU EM VALOR
    if trigger_vl_caiu != 0:
        print(f'Vai validar se a ação {acao} caiu R${trigger_vl_caiu}')
        print(f'A ação {acao} variou {real_vl_variacao}%')

        print(trigger_vl_caiu*-1)
        print(real_vl_variacao)
        
        if (trigger_vl_caiu*-1 >= real_vl_variacao):
        
            print('--->>> Gatilho ativado')

            mensagem = f'''
                <b>ATENÇÃO</b>
                <i>avisa_acoes BOT Informa</i>

                <u>Sua ação <b>{acao}</b> caiu <b> R$ {real_vl_variacao}</b>  </u> 
                <pre>Trigger ->R$ {trigger_vl_caiu*-1} de queda </pre>
            
            '''
        
            send_message_telegram(trigger_ids_chat,mensagem)         
            # se disparou uma mensagem por gatilho, não valida os demais gatilhos
            continue

{'acao': 'ITUB4.SA', 'percent_caiu': 1, 'percent_subiu': 0.15, 'vl_caiu': 1, 'vl_subiu': 0.2, 'ids_chat': [1103869584]}
[*********************100%%**********************]  1 of 1 completed
problema no yfinance
[*********************100%%**********************]  1 of 1 completed
############################
Vai validar se a ação ITUB4.SA caiu 1%
A ação ITUB4.SA variou -0.6%
-1
-0.6
############################
Vai validar se a ação ITUB4.SA subiu 0.15%
A ação ITUB4.SA variou -0.6%
0.15
-0.6
Vai validar se a ação ITUB4.SA subiu R$0.2
A ação ITUB4.SA variou -0.21%
0.2
-0.21
Vai validar se a ação ITUB4.SA caiu R$1
A ação ITUB4.SA variou -0.21%
-1
-0.21
{'acao': 'BBDC4.SA', 'percent_caiu': 0.5, 'percent_subiu': 0.5, 'vl_caiu': 1.5, 'vl_subiu': 0.15, 'ids_chat': [1103869584]}
[*********************100%%**********************]  1 of 1 completed
problema no yfinance
[*********************100%%**********************]  1 of 1 completed
############################
Vai validar se a ação BBDC4.SA 